## Advent of code 2023 day 1-10
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv environment (which is set up with pypy3.10-v7.3.13-win64)
# to activate it from a git bash shell: source .venv/Scripts/activate

import collections
import itertools
import functools
import re
import copy
import math
import sys
import time
import json
import heapq
import bisect
import random
import sortedcontainers

import zio

In [ ]:
# version check and timestamp
# NB the timestamp supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}')

In [ ]:
# 2023 day 3
# mv ~/Downloads/input* data_src/2023-day-3-input.txt
# big input file looks like: a map?
# idea: part 1 parse as basic map, then char by char left to right build up numbers while
#  scanning for symbols around the number chars and recording them
# part 2: invert the list of numbers to a map of symbols, then iterate over that

sample2='''
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
'''

def gen_numbers(lines):
    '''parse out numbers with their symbols, return list of lists, first is
    number, followed by a set of symbols with their coords'''
    res=[]
    curnum=''
    cursyms=set()
    for y0, row in enumerate(lines):
        for x0,c in enumerate(row):
            if '0' <= c <= '9':
                curnum+=c
                for y in range(y0-1, y0+2):
                    for x in range(x0-1, x0+2):
                        if y>=0 and y<len(lines) and x>=0 and x<len(lines[y]):
                            potsym=lines[y][x]
                            if not (potsym=='.' or ('0' <= potsym <= '9')):
                                potsymtup=(potsym, x, y)
                                cursyms.add(potsymtup)
            else:
                if len(curnum)>0:
                    res.append( [int(curnum), cursyms] )
                    curnum=''
                    cursyms=set()
        if len(curnum)>0:
            res.append( [int(curnum), cursyms] )
            curnum=''
            cursyms=set()
    return res

sample1=open('data_src/2023-day-3-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

# part 1
numlist=gen_numbers(lines)
totalparts=sum([ tup[0] for tup in numlist if len(tup[1]) > 0 ])
print(f'part 1: {totalparts=}')

# part 2
starmap={} # maps startup (sym, x, y) to set of surrounding nums
           #  (part numbers, which are in this context unique per symbol)
for tup in numlist:
    num=tup[0]
    for symtup in tup[1]:
        if symtup[0] != '*':
            continue
        ls=starmap.setdefault(symtup, set())
        ls.add(num)
starsum=0
for k,v in starmap.items():
    if len(v)==2:
        v=list(v)
        starsum+=v[0]*v[1]
print(f'part 2: {starsum=}')

In [ ]:
# 2023 day 2
# mv ~/Downloads/input* data_src/2023-day-2-input.txt
# big input file looks like: 100 games with color counts
# idea: part 1 parse by descending splits (runs in games, elements in runs),
#  then check the run limits along the way
# part 2: again iterate, now calculating the minimum (actually maximum) per game

sample2='''
Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
'''

sample1=open('data_src/2023-day-2-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

# part 1
data=[]
okgameids=set()
maxgam1={'red': 12, 'green': 13, 'blue': 14}
for s in lines:
    game={}
    tup=s.split(':')
    game['id']=int(tup[0][5:])
    okgameids.add(game['id'])
    tup=tup[1].split(';')
    runs=[]
    for rn in tup:
        rund={}
        elems=rn.split(',')
        for elem in elems:
            pair=elem.split()
            color=pair[1]
            cnt=int(pair[0])
            rund[color]=cnt
            if color in maxgam1:
                maxcnt=maxgam1[color]
                if cnt>maxcnt:
                    okgameids.discard(game['id'])
        runs.append(rund)
    game['runs']=runs
    data.append(game)
print(f'part 1: {sum(okgameids)}')

# part 2
powsum=0
for game in data:
    mincnt={}
    for run in game['runs']:
        for k,v in run.items():
            assert k in {'red', 'green', 'blue'}
            if k in mincnt:
                mincnt[k]=max(v, mincnt[k])
            else:
                mincnt[k]=v
    pow=1
    for v in mincnt.values():
        pow*=v
    powsum+=pow
print(f'part 2: {powsum=}')

In [ ]:
# 2023 day 1
# mv ~/Downloads/input* data_src/2023-day-1-input.txt
# big input file looks like: all kinds of words with digits in them
# idea: part 1 parse as lines, then iterate over s and reversed(s) to find digits
# part 2: map of digit words, then for each position check digit and digit words

sample2='''
1abc2
pqr3stu8vwx
a1b2c3d4e5f
treb7uchet
'''

sample3='''
two1nine
eightwothree
abcone2threexyz
xtwone3four
4nineeightseven2
zoneight234
7pqrstsixteen
'''

sample1=open('data_src/2023-day-1-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

# part 1
n=0
for s in lines:
    digits=''
    for c in s:
        if '0' <= c <= '9':
            digits+=c
            break
    for c in reversed(s):
        if '0' <= c <= '9':
            digits+=c
            break
    if len(digits)!=2:
        continue
    n+=int(digits)
print(f'part 1: {n=}')

# part 2
digmap={
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5,
    'six': 6,
    'seven': 7,
    'eight': 8,
    'nine': 9,
}
n=0
for s in lines:
    digits=''
    for i in range(len(s)):
        c=s[i]
        if '0' <= c <= '9':
            digits+=c
        else:
            for k, v in digmap.items():
                if s[i:].startswith(k):
                    digits+=str(v)
                    break    
    n+=int(digits[0]+digits[-1])
print(f'part 2: {n=}')

In [ ]:
# TEMPLATE
# 2023 day 1
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2023-day-1-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample2='''

'''

sample1=open('data_src/2023-day-1-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
groups=zio.get_line_groups(sample1.splitlines(), nostrip=False)
data=[ int(s) for s in lines[0].split(',') ]
data=[ s.split() for s in lines ]
data=[ [cmd, int(num), 0] for cmd, num in data ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed